# Using EcoFOCIpy to process raw field data

## DY2410 (Fall Mooring - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2024/CTDcasts/dy2410/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2410' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2410.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD001.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD003.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD004.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD005.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD006.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD007.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD009.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD010.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD011.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD012.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD013.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD014.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD015.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD016.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD017.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD018.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD019.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD020.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD021.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD022.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD023.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD024.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD025.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD026.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD027.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD028.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD029.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD030.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD031.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD032.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD033.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD034.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD035.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD036.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD037.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD038.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD039.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD040.btl


...

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD054.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD055.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD056.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD057.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD058.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD059.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD060.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD061.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD062.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD063.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD064.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD065.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD066.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD067.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD068.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD070.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD071.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD072.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD073.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD075.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD076.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD077.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD078.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD079.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD082.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD083.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2410/rawconverted/CTD096.btl


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['CTD001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,par,scan,datetime
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11
mean,6.017800,87.375000,262.101000,32.399182,32.402082,25.387200,25.389545,5.699182,82.748909,248.223636,0.509182,6.873727,6.873345,3.186000,1.739964,10485.181818,2024-09-03 21:48:55.818181632
min,6.012700,87.301000,261.880000,32.398600,32.401400,25.386600,25.388900,5.674100,82.384000,247.130000,0.379300,6.872600,6.872400,3.054000,1.691000,10319.000000,2024-09-03 21:48:49
25%,6.015750,87.344000,262.011500,32.398900,32.401950,25.386950,25.389400,5.690500,82.623500,247.846500,0.436400,6.873400,6.872750,3.140500,1.721300,10408.500000,2024-09-03 21:48:52.500000
50%,6.017100,87.364000,262.070000,32.399300,32.402200,25.387300,25.389600,5.703100,82.806000,248.396000,0.497400,6.873700,6.873400,3.214000,1.740900,10486.000000,2024-09-03 21:48:56
75%,6.019950,87.406000,262.195500,32.399400,32.402300,25.387400,25.389800,5.709650,82.902000,248.679000,0.581300,6.874250,6.873700,3.218000,1.757450,10564.500000,2024-09-03 21:48:59
max,6.022900,87.451000,262.324000,32.399700,32.402500,25.387700,25.390000,5.719300,83.042000,249.100000,0.622100,6.874600,6.874500,3.291000,1.785400,10644.000000,2024-09-03 21:49:02
std,0.003266,0.047921,0.142396,0.000349,0.000349,0.000355,0.000370,0.015072,0.219222,0.656222,0.083482,0.000674,0.000709,0.074551,0.026261,105.596229,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2410',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2024, 9, 3),
 'EndDate': datetime.date(2024, 9, 28),
 'Project': 'FOCI',
 'ChiefScientist': 'Ryan McCabe',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Kodiak, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'EcoFOCI Fall Mooring Cruise',
 'CruiseYear': 2024,
 'ctdlogs_pdf_name': 'DY2410_CTD_logs_edit.pdf',
 'Accession': '',
 'Links': '\r',
 'QC_Status': '',
 'QC_Signature': ''}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54860,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2410',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2410',
 'Project': 'Fall Mooring Cruise',
 'StationNo_altname': 's1h1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 53,
 'LatitudeMin': 37.867,
 'LongitudeDeg': 167,
 'LongitudeMin': 23.641,
 'GMTDay': 4,
 'GMTMonth': 'Sep',
 'GMTYear': 2024,
 'GMTTime': 20760,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 95,
 'StationNameID': 'UM01',
 'MaxDepth': 90,
 'InstrumentSerialNos': '',
 'Notes': 'Umnak Mooring.  Popping bottles @ surface to test and get filtered water for chlor. rig',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'Chlor

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['CTD001.btl'] = cruise_data['CTD001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['CTD001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,Pressure [dbar],par,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
6.0,6.0139,87.318,261.929,32.3993,32.402,25.3873,25.3894,5.6957,82.699,248.073,0.5627,6.8737,6.8736,3.291,1.718,10486.0,2024-09-03 21:48:56,ctd001


In [10]:
cruise_data['CTD001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'Pressure [dbar]', 'par', 'scan',
       'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['CTD001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 11)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 6.017 6.023 6.019 ... 6.017 6.02 6.016
    oxy_percentsat_ch1  (bottle) float64 87.36 87.45 87.4 ... 87.36 87.4 87.34
    oxy_conc_ch1        (bottle) float64 262.1 262.3 262.2 ... 262.1 262.2 262.0
    salinity_ch1        (bottle) float64 32.4 32.4 32.4 32.4 ... 32.4 32.4 32.4
    salinity_ch2        (bottle) float64 32.4 32.4 32.4 32.4 ... 32.4 32.4 32.4
    sigma_t_ch1         (bottle) float64 25.39 25.39 25.39 ... 25.39 25.39 25.39
    ...                  ...
    temperature_ch2     (bottle) float64 6.875 6.874 6.874 ... 6.872 6.872 6.872
    Pressure [dbar]     (bottle) float64 3.22 3.216 3.214 ... 3.137 3.054 3.176
    par                 (bottle) float64 1.733 1.741 1.753 ... 1.755 1.76 1.723
    scan                (bottle) float64 1.032e+04 1.036e+04 ... 1.064e+04
    datetime            (bottle) datetime64[ns] 2024-09-03T21:48:49 ... 2024-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['CTD001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 11, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.017 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 87.36 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 262.1 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.4 ... ...
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.4 ... ...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.39 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 6.875 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 3.22 ... ...
    par                 (latitude, longitude, time, bottle) float64 1.733 ......
    scan                (latitude, longitude, time, bottle) float64 1.032e+04...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 11, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.017 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 87.36 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 262.1 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.4 ... ...
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.4 ... ...
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.39 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 6.875 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 3.22 ... ...
    par                 (latitude, longitude, time, bottle) float64 1.733 ......
    scan                (latitude, longitude, time, bottle) float64 1.032e+04...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/48)
    CruiseID:                      DY2410
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-11-13T08:10:30Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
cruise_data[cast] = cruise_data[cast].rename(columns={
                    't090c':'temperature_ch1',
                    't190c':'temperature_ch2',
                    'sal00':'salinity_ch1',
                    'sal11':'salinity_ch2',
                    'sbox0mm/kg':'oxy_conc_ch1',
                    'sbeox0ml/l':'oxy_concM_ch1',
                    'sbox1mm/kg':'oxy_conc_ch2',
                    'sbeox1ml/l':'oxy_concM_ch2',
                    'sbeox0ps':'oxy_percentsat_ch1',
                    'sbeox1ps':'oxy_percentsat_ch2',
                    'sigma-t00':'sigma_t_ch1',
                    'sigma-t11':'sigma_t_ch2',
                    'cstarat0':'Attenuation',
                    'cstartr0':'Transmittance',
                    'fleco-afl':'chlor_fluorescence',
                    'turbwetntu0':'turbidity',
                    'prdm':'pressure',
                    'empty':'empty', #this will be ignored
                    'flag':'flag'})

cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                            instrument_yaml=inst_config, 
                            operation_yaml=cruise_config,
                            operation_type='ctd')

cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

#add global attributes
cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cast_label = cast.lower().split('d')[-1].split('.')[0]
cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                           filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


In [19]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history